# 1. load corpus

# 2. TFIDF

## 2.1. term frequency

## 2.2. document frequency

# 3. word cloud

# 4. spread key words

# 5. NER

## 5.1. Stanford CoreNLP

## 5.2. HIT LPT

## 5.3. Jieba

# 6. POS and Dependency parseing